In [3]:
import pandas as pd
import random
import numpy as np

### Importando os dados da nossa tabela

In [4]:
df = pd.read_excel('Tabela_QM9_Termodinamica.xlsx')

In [44]:
df.head(5)

,Fórmula,N° de Carbonos,N° de Hidrogênios,N° de Oxigênios,N° de Nitrogênios,N° de Fósforos,Isomeric Smiles,Smiles,Massa Molar (g/mol),Entalpia de Combustão (kcal/mol),Entalpia de Formação (kcal/mol),Energia Interna (kcal),Energia Livre de Gibbs (kcal)
0,CH4,1,4,0,0,0,C,C,16.043,239.315162,401.015030,398.643671,372.472128
1,H3N,0,3,0,1,0,N,N,17.031,0.000000,280.399527,278.620537,259.339050
2,H2O,0,2,1,0,0,O,O,18.015,0.000000,215.159864,213.974499,201.407364
3,C2H2,2,2,0,0,0,C#C,C#C,26.038,246.164372,389.016419,387.238057,365.801074
4,CHN,1,1,0,1,0,C#N,C#N,27.026,0.000000,304.091780,302.907042,288.720305


### Funções Para algoritmo

In [18]:
def gene_combustivel():
    """ Busca aleatória dentro da base de dados um combustivel
    Retorna:
        Um compsoto escolhido aleatóriamente pelo seu índice, nosso gene
    """
    #fazendo lista dos índices dos nossos compostos
    lista_indices = df.index.tolist()
    # Escolhendo um índice aleatório dentro da lista de índices
    indice_aleatorio = random.choice(lista_indices)
    # indica qual composto corresponde ao indice aleatório escolhido
    # composto = df.loc[indice_aleatorio, 'Smiles']
    composto = indice_aleatorio
    return composto

In [50]:
def individuo_combustivel():
    """Pega um gene aleatóriamente
    Retorna:
        um combustivel
    """
    individuo = []
    for _ in range(2):
        individuo.append(gene_combustivel())
    return individuo

In [51]:
individuo_combustivel()

[50521, 84170]

In [21]:
def pop_posto(tamanho):
    """Agrupa os individuos para formar a população de dupla de combustíveis (posto)
    Argumentos:
        tamanho: tamanho da população, quantos individuos a compõe
    Retorna:
        população com número de individuos escolhidos
    """
    posto = []
    for _ in range(tamanho):
        posto.append(individuo_combustivel())
    return posto
    

In [22]:
def selecao_combustivel(populacao, fitness):
    """Seleciona individuos de uma população usando o método da roleta.
    Nota: apenas funciona para problemas de maximização.
    Args:
      populacao: lista com todos os individuos da população
      fitness: lista com o valor da funcao objetivo dos individuos da população
    Returns:
      População dos indivíduos selecionados.
    """
    populacao_selecionada = random.choices(
        populacao, weights=fitness, k=len(populacao)
    )
    return populacao_selecionada

In [60]:
def obj_posto_avaliacao(individuo):
    """ Calcular o fitness do individuo, usando uma função matemática
    Argumentos:
        individuo: lista com dois combustiveis selecionados aleatoriamente
    Retorna:
        O fitness de cada um indivíduo
    """
    # separa os genes dentro do individuo, para avaliar separadamente
    # pega os seus indices
    comb1 = individuo[0]
    comb2 = individuo[1]
    # Pega a informação de entalpia de cada um dos combustiveis
    Hc1 = df.loc[comb1, 'Entalpia de Combustão (kcal/mol)']
    Hc2 = df.loc[comb2, 'Entalpia de Combustão (kcal/mol)']
    #Hc = df.loc[individuo, 'Entalpia de Combustão (kcal/mol)']
    # Pega a massa molar de cada um dos combustiveis
    Mmol1 = df.loc[comb1, 'Massa Molar (g/mol)']
    Mmol2 = df.loc[comb2, 'Massa Molar (g/mol)']
    #Mmol = df.loc[individuo, 'Massa Molar (g/mol)']
    
    # Faz função afim para avaliar o valor da entalpia
    y1 = 2*Hc1
    y2 = 2*Hc2
    # y = 2* Hc
    
    # Faz função para avaliar o valor da massa molar
    g1 = 10000/Mmol1
    g2 = 10000/Mmol2
    #g = 1/Mmol
    
    # junta essas informações
    fitness = y1+g1+y2+g2
    
    return  fitness

In [61]:
def func_obj_pop_posto(populacao):
    """Calcula a funcao objetivo para todos os membros de uma população
    Args:
      populacao: lista com todos os individuos da população
    Return:
      Lista de valores represestando a fitness de cada individuo da população.
    """
    fitness = []
    for individuo in populacao:
        fobj = obj_posto_avaliacao(individuo)
        fitness.append(fobj)
    return fitness

In [66]:
def mutacao_posto(individuo):
    """Muda o individuo 
    """
    gene_a_ser_mutado = random.randint(0, len(individuo) - 1)
    individuo[gene_a_ser_mutado] = gene_combustivel()
    
    return individuo

In [64]:
def cruzamento_posto(pai, mae):
    """Operador de cruzamento de ponto simples.

    Args:
      pai: uma lista representando um individuo
      mae : uma lista representando um individuo

    Returns:
      Duas listas, sendo que cada uma representa um filho dos pais que foram os
      argumentos.
    """
    ponto_de_corte = random.randint(1, len(pai) - 1)

    filho1 = pai[:ponto_de_corte] + mae[ponto_de_corte:]
    filho2 = mae[:ponto_de_corte] + pai[ponto_de_corte:]

    return filho1, filho2

In [56]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_MUTACAO = 0.05
CHANCE_CRUZAMENTO = 0.5

In [57]:
# função local
def cria_populacao_inicial(TAMANHO_POP):
    return pop_posto(TAMANHO_POP)
def funcao_objetivo_pop(populacao):
    return func_obj_pop_posto(populacao)
def funcao_objetivo_individuos():
    pass
def funcao_selecao(populacao, fitness):
    return selecao_combustivel(populacao, fitness)
def funcao_mutacao(individuo):
    return mutacao_posto(individuo)
def funcao_cruzamento(pai, mae):
    return cruzamento_posto(pai, mae)

Algoritmo genético

In [91]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = 0
hall_da_fama = []

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    
    if maior_fitness > melhor_fitness_ja_visto:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        #composto = df.loc[melhor_individuo_ja_visto, 'Smiles']
        #entalpia = df.loc[melhor_individuo_ja_visto, 'Entalpia de Combustão (kcal/mol)']
        #massa = df.loc[melhor_individuo_ja_visto, 'Massa Molar (g/mol)']
        #print(melhor_individuo_ja_visto)
        hall_da_fama.append(melhor_individuo_ja_visto)
        #print(f"Os melhores combustiveis {composto}")
        #print()
        #print(f' massa = {massa} ')
        #print()
        #print(f' entalpia = {entalpia}')
        
print(hall_da_fama)
print()

for i in hall_da_fama:
    for j in range(2):
        mol = i[j]
        composto = df.loc[mol, 'Smiles']
        entalpia = df.loc[mol, 'Entalpia de Combustão (kcal/mol)']
        massa = df.loc[mol, 'Massa Molar (g/mol)']
        print(i, mol, composto, entalpia, massa)
        print()

[[105232, 106671], [105232, 96344], [105232, 11028], [81700, 11028]]

[105232, 106671] 105232 CCC1(C(C)C)OC1C 1082.931139 128.215

[105232, 106671] 106671 CC1OC2(CO)CC12O 534.757065 130.143

[105232, 96344] 105232 CCC1(C(C)C)OC1C 1082.931139 128.215

[105232, 96344] 96344 CC1(O)COCCC1O 584.475278 132.159

[105232, 11028] 105232 CCC1(C(C)C)OC1C 1082.931139 128.215

[105232, 11028] 11028 CC1C=CC2(C)CC12 1002.315609 108.184

[81700, 11028] 81700 CC1C2CCCCC1C2 1230.67257 124.227

[81700, 11028] 11028 CC1C=CC2(C)CC12 1002.315609 108.184

